In [1]:
!pip install opencv-python

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np

from collections import defaultdict

In [2]:
def writeBounds(file_name, frames_bounds):
    import os.path
    if not os.path.isfile(file_name):
        with open(file_name, 'w') as f:    
            f.write('frame,x1,y1,x2,y2,n_bound\n')
    
    with open(file_name, 'a') as f:    
        for frame_n in frames_bounds:
            for bound in frames_bounds[frame_n]:
                f.write(str(frame_n) + ',' + ','.join([str(x) for x in bound]) + '\n')


def openBounds(file_name):
    frames_bounds = defaultdict(list)
    with open(file_name, 'r') as f:
        bound_n = 0
        for line in f.readlines()[1:]:
            values = [int(x) for x in line.split(',')]
            frames_bounds[values[0]].append(values[1:])
            bound_n = values[-1]
    return frames_bounds, (bound_n + 1)
            

def showBouds(frame_n, frame, window_name, frame_bounds):
    for bound in frame_bounds[frame_n]:
        cv2.rectangle(frame, tuple(bound[0:2]), tuple(bound[2:4]), (255,0,0), 2)
        cv2.putText(frame, 'BOUND: ' + str(bound[4]) , tuple(bound[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (255, 255, 255), 1)

        showFrame(frame, window_name, frame_n)        

def showFrame(frame, window_name, frame_n):    
    cv2.putText(frame, str(frame_n) , (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    cv2.imshow(window_name, frame)

In [3]:
def normalize(arr):
    arr =  arr.astype('float')
    
    minval = arr.min()
    maxval = arr.max()

    if minval != maxval:
        arr -= minval
        arr *= (255/(maxval-minval))

    return arr

def detectGreensRGBGray(frame):
    [height, width, _] = frame.shape
    
    red = frame[:,:,0].astype('float')/255
    green = frame[:,:,1].astype('float')/255
    blue = frame[:,:,2].astype('float')/255
    
    zeros = np.zeros((height, width), np.float)  
    
    result= cv2.max(green-blue, zeros)
    return normalize(cv2.max(result, zeros)*255).astype('uint8')

def detectGreensHSVGrayMask(frame):
    [height, width, _] = frame.shape
    
    notMaskValue = 0.5
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, (10, 25, 25), (120, 255, 255))        

    imask = mask > 0
    result = np.zeros((height, width, 3), np.uint8)       
    
    mask = imask.astype('uint8') * 255
        
    kernel = np.ones((2,2),np.uint8)
    mask = cv2.erode(mask, kernel, iterations = 2)
    mask = cv2.dilate(mask, kernel, iterations = 2)
    
    mask = np.where(mask == 0, notMaskValue, 1) 
    
    result[:,:,0] = np.multiply(frame[:,:,0], mask).astype('uint8')
    result[:,:,1] = np.multiply(frame[:,:,1], mask).astype('uint8')
    result[:,:,2] = np.multiply(frame[:,:,2], mask).astype('uint8')
    
    return result

def detectGreensHSVMaks(frame):
    [height, width, _] = frame.shape
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)

    mask = cv2.inRange(hsv, (10, 25, 25), (120, 255, 255))        
    imask = mask > 0
    result = np.zeros((height, width, 3), np.uint8)   
    result[imask] = frame[imask]
    
    return result

In [4]:
class RingBuffer:
    def __init__(self, size):
        self.data = [None for i in range(size)]

    def append(self, x):
        self.data.pop(0)
        self.data.append(x)

    def get(self):
        return self.data  
    
class RACapWrapper:
    def __init__(self, cap, mlenght):
        self.cap = cap
        self.mlenght = mlenght
        self.frameHistory = RingBuffer(mlenght)
        self.actualFrame = -1    
    
    def readAt(self, frameN):       
        if frameN == self.actualFrame:
            return self.frameHistory.get()[-1].copy(), frameN
        elif frameN > self.actualFrame:
            if self.cap.isOpened():
                _, frame = self.cap.read()
                self.frameHistory.append(frame)
                self.actualFrame += 1
                
                return self.readAt(frameN)
            else:
                raise Exception("Cap is close")
        elif frameN < self.actualFrame:
            if (self.actualFrame - frameN) >= self.mlenght:
                frameN = self.actualFrame - self.mlenght + 1
            
            return self.frameHistory.data[frameN - self.actualFrame - 1].copy(), frameN

In [7]:
window_name = "Ventana de etiquetado"

def mouseEvent(event, x, y, flags, params):    
    global draw_state, first_corner
    global bounds, bound_n
    global frame, frame_n
    global window_name
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if draw_state == 0:
            draw_state = 1
            
            first_corner = (x, y)
            cv2.circle(frame, first_corner, 4, (0, 0, 255), 1)
            
            showFrame(frame, window_name, frame_n)
        elif draw_state == 1:
            draw_state = 0
            
            frames_bounds[frame_n].append(list(first_corner) + [x, y, bound_n])                     
            bound_n += 1
            
            cv2.rectangle(frame, first_corner, (x, y), (0,255,0), 2)            
            cv2.circle(frame, (x, y), 4, (0, 0, 255), 1)
            cv2.putText(frame, 'BOUND: ' + str(bound_n - 1) , first_corner, cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (255, 255, 255), 1)            
            
            showFrame(frame, window_name, frame_n)
        else:
            draw_state = 0        


frame_init_offset = 370
frame_count = 1700 - frame_init_offset

draw_state = 0
first_corner = (0, 0)
frame = None
frame_height = 420 #para recortar la fecha y hora

file_name = 'bounds2.txt'

open_file = True
if open_file:
    last_frames_bounds, bound_n = openBounds(file_name) 
else:
    last_frames_bounds, bound_n = defaultdict(list), 0

frames_bounds = defaultdict(list)
frame_n = 0

cap = cv2.VideoCapture('../DCIM/saturated-noaudio.avi')

cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, mouseEvent)

racap = RACapWrapper(cap, 30)
frame_n = frame_init_offset

while frame_n < (frame_init_offset + frame_count):
    frame, frame_n = racap.readAt(frame_n)
    frame = frame[:frame_height,:]  
        
    showFrame(frame , "Original frame", frame_n)        
    
    frame = detectGreensHSVGrayMask(frame)
    showFrame(frame , window_name, frame_n)         

    showBouds(frame_n, frame, window_name, last_frames_bounds)
    showBouds(frame_n, frame, window_name, frames_bounds)

    key = cv2.waitKey(-1) 
    if key == 27:
        break;
    else:        
        frame_n += (-1 if key == 97 else 1)

writeBounds(file_name, frames_bounds)
            
cap.release()

cv2.destroyAllWindows()

In [ ]:
#frame = cv2.resize(frame, (0, 0), fx = 0.5, fy = 0.5, interpolation=cv2.INTER_NEAREST)